# Initialize qcodes

In [1]:
import qcodes as qc
from qcodes.dataset.sqlite.database import initialise_or_create_database_at
import IPython.lib.backgroundjobs as bg
from plottr.apps import inspectr
from qcodes.instrument.parameter import Parameter
from qc_measure_20240711 import *
print_versions()

qcodes: 0.46.0
qcodes_contrib_drivers: 0.22.0
plottr: 0.13.0
python: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]
matplotlib: 3.8.4
numpy: 1.26.4
scipy: 1.14.0


Update db_path here:

In [2]:
# Create a new db file name for each cooldown.
db_path = 'test 20240707-XL105 20221222-InAs2DEGJJ15 username.db'

qc.logger.start_all_logging()
initialise_or_create_database_at(db_path)

station = qc.Station()

# Open Plottr
# jobs = bg.BackgroundJobManager()
# jobs.new(inspectr.main, db_path)

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\Measurement\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\Measurement\.qcodes\logs\240712-11196-qcodes.log


# Load instruments

In [3]:
from qcodes_contrib_drivers.drivers.QDevil import QDAC2
# qdac.close()
qdac = QDAC2.QDac2('QDAC',  address= 'COM4')

from qcodes.instrument_drivers.Keysight.Keysight_34461A_submodules import Keysight_34461A
# dmm.close()
dmm =  Keysight_34461A('Keysight_34461A', 'USB0::0x2A8D::0x1301::MY60060848::INSTR')

Connected to: QDevil QDAC-II (serial:78, firmware:7-0.17.5) in 0.14s
Connected to: Keysight Technologies 34461A (serial:MY60060848, firmware:A.03.03-03.15-03.03-00.52-04-03) in 0.16s


# Configure Parameters and station

In [4]:
def get_V_measure():
    return dmm.volt()

V_measure = Parameter('V_measure', unit='V', label='Vmeas', 
                                 get_cmd=get_V_measure, set_cmd=None)


Dummy_x = DummyParameter(name='dummy_x',unit='')

Dummy_y = DummyParameter(name='dummy_y',unit='')

Vg = qdac.ch01.dc_constant_V

In [5]:
# First remove all components
st_components = list(station.components.keys())
for i in st_components:
    station.remove_component(i)

# Add components
for i in [dmm, V_measure, Vg]:
    print(i)
    station.add_component(i)

<Keysight_34461A: Keysight_34461A>
V_measure
QDAC_ch01_dc_constant_V


# Begin scans

## 1d

In [6]:
qm = QMeasure(exp_name='Ibias',sample_name='left-12-13-a',export_dat=True, mute_qclient=False)
qm.set_parameters_to_aquire(para_list=[V_measure], parallel=True)
qm.set_parameters_in_measurement_name(para_list=[V_measure, dmm.volt])

qm.set_scan_parameter('x', para=Dummy_x, start=0, stop=1, points=101, delay=0.01)
# qm.set_scan_parameter('y', para=None, start=None, stop=None, points=None, delay=0)
qm.scan2d(bwd=True)

Starting experimental run with id: 9. 
Starting experimental run with id: 10. 


Total: 0 min 9 s. Each point: 0.042 s.
DAT and SET file saved: DAT\Ibias.left-12-13-a.9
DAT and SET file saved: DAT\Ibias.left-12-13-a.10


In [7]:
qm = QMeasure(exp_name='Ibias',sample_name='left-12-13-a',export_dat=True, mute_qclient=False)
qm.set_parameters_to_aquire(para_list=[V_measure], parallel=True)
qm.set_parameters_in_measurement_name(para_list=[V_measure, dmm.volt])

qm.set_scan_parameter('x', para=Vg, start=0, stop=0.1, points=101, delay=0.01)
# qm.set_scan_parameter('y', para=None, start=None, stop=None, points=None, delay=0)
qm.scan2d(bwd=True)

Starting experimental run with id: 11. 
Starting experimental run with id: 12. 


Total: 0 min 10 s. Each point: 0.042 s.
DAT and SET file saved: DAT\Ibias.left-12-13-a.11
DAT and SET file saved: DAT\Ibias.left-12-13-a.12


## 2d

In [8]:
qm = QMeasure(exp_name='Ibias',sample_name='left-12-13-a',export_dat=True, mute_qclient=False)
qm.set_parameters_to_aquire(para_list=[V_measure], parallel=True)
qm.set_parameters_in_measurement_name(para_list=[V_measure, dmm.volt])

qm.set_scan_parameter('x', para=Vg, start=0, stop=0.1, points=101, delay=0.01)
qm.set_scan_parameter('y', para=Dummy_y, start=0, stop=1, points=10, delay=0.1)
qm.scan2d(bwd=True)

Starting experimental run with id: 13. 
Starting experimental run with id: 14. 


Total: 1 min 28 s. Each point: 0.044 s.
DAT and SET file saved: DAT\Ibias.left-12-13-a.13
DAT and SET file saved: DAT\Ibias.left-12-13-a.14


## Export data manually

In [10]:
d2d = Db2Dat()
exp_id = 14
dat_folder = r'.\DAT'
dat_path = d2d.to_dat(db_path, exp_id, dat_folder,overwrite=False)

File ".\DAT\Ibias.left-12-13-a.14.dat" already exists and has not been overwritten.
File ".\DAT\Ibias.left-12-13-a.14.set" already exists and has not been overwritten.
DAT and SET file saved: .\DAT\Ibias.left-12-13-a.14


# Special scans: fast1d

In [11]:
from qc_fast1d_20240710 import *

In [19]:
def Ibias_to_Vdac(x):
    y =  x/1.
    return y

# You may want to modify the output_channel and nplc
TRG_DAC = TrigQDAC(qdac,ch=1,trg_out=1,trg_in=1,trg_internal=1,)
TRG_DMM = TrigDMM(dmm,nplc=0.2) 

def reset_trig_modes():
    '''
    Reset trig modes of dac and dmm
    '''
    TRG_DAC.back_to_no_trig()
    TRG_DMM.back_to_auto_trig()
    TRG_DMM.dmm.NPLC()
    
class QMeasureFast(QMeasure):
    '''
    A new classs replacing _scan1d in QMeasure, for fast scan with triggers
    '''  
    def _scan1d(self,datasaver, list_x, y):
        para_x = self.scan_para['x']['parameter']
        delay_x = self.scan_para['x']['delay']
        parameters_to_fetch = self.monitor_para['list']
        para_y = self.scan_para['y']['parameter']

        list_v = Ibias_to_Vdac(list_x)
        # make sure that parameters_to_fetch == [Vmeasure]
        p_fetch = parameters_to_fetch[0]

        if self.counter==0:
            # Configure the DMM buffer and trigger
            TRG_DMM.initialize_meas(list_v,trig_delay=delay_x)
            # This will upload the sweeping list and configure triggers
            TRG_DAC.initialize_sweep(list_v)
            self.counter += 1

        # Send the initial trigger
        TRG_DMM.enter_meas()
        TRG_DAC.fire_sweep() 

        try:
            d = TRG_DMM.fetch()
        except KeyboardInterrupt:
            raise(KeyboardInterrupt)

        d = d[1:]
        list_val = [[(p_fetch, i)] for i in d]

        for x, val in zip(list_x, list_val):
            rslt = [(para_x, x), (para_y, y)] + val
            datasaver.add_result(*rslt)

            if self.is_fwd_now or self.scan_dim==1:
                self.qclient.add_data([i[1] for i in rslt])
        self.qclient.update_plot()

In [22]:
qm_fast = QMeasureFast(exp_name='Ibias',sample_name='left-12-13-a',export_dat=True, mute_qclient=False)
# Ibias_fast must be a dummy parameter, not the real Ibias parameter, only the parameter name will be used
Ibias_fast = DummyParameter(name='Ibias_fast', unit='uA')

qm_fast.set_parameters_to_aquire(para_list=[V_measure], parallel=True)
qm_fast.set_parameters_in_measurement_name(para_list=[V_measure, dmm.volt])

qm_fast.set_scan_parameter('x', para=Ibias_fast, start=0, stop=1, points=101, delay=0.01)
# qm_fast.set_scan_parameter('y', para=Dummy_y, start=0, stop=1, points=2, delay=0)
qm_fast.scan2d(bwd=True)

reset_trig_modes()

Starting experimental run with id: 25. 
Starting experimental run with id: 26. 
Total: 0 min 4 s. Each point: 0.014 s.
DAT and SET file saved: DAT\Ibias.left-12-13-a.25
DAT and SET file saved: DAT\Ibias.left-12-13-a.26
Setting qdac back to no trigger and fixed voltage...
Setting dmm back to auto trig mode (if failed, run it once more)...


In [23]:
qm_fast = QMeasureFast(exp_name='Ibias',sample_name='left-12-13-a',export_dat=True, mute_qclient=False)
# Ibias_fast must be a dummy parameter, not the real Ibias parameter, only the parameter name will be used
Ibias_fast = DummyParameter('Ibias_fast',unit='uA')

qm_fast.set_parameters_to_aquire(para_list=[V_measure], parallel=True)
qm_fast.set_parameters_in_measurement_name(para_list=[V_measure, dmm.volt])

qm_fast.set_scan_parameter('x', para=Ibias_fast, start=0, stop=1, points=101, delay=0)
qm_fast.set_scan_parameter('y', para=Dummy_y, start=0, stop=1, points=10, delay=0)
qm_fast.scan2d(bwd=True)

reset_trig_modes()

Starting experimental run with id: 27. 
Starting experimental run with id: 28. 


Total: 0 min 9 s. Each point: 0.004 s.
DAT and SET file saved: DAT\Ibias.left-12-13-a.27
DAT and SET file saved: DAT\Ibias.left-12-13-a.28
Setting qdac back to no trigger and fixed voltage...
Setting dmm back to auto trig mode (if failed, run it once more)...


## Reset trigger mode

In [24]:
reset_trig_modes()

Setting qdac back to no trigger and fixed voltage...
Setting dmm back to auto trig mode (if failed, run it once more)...
